In [17]:
#### load packages, *.h5 folder and exp.h5 file location
%matplotlib widget

In [18]:
from essential_func import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model



qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

samples_csv                = 'data_directory.csv'
default_sample_dir         = 'Mar-2023-Sample#1898'       # 'July-2021-Sample#6'    '21-Nov'   'July-sorted'   '20-Dec'    '22-Oct'   'Oct-2022-1971'   'Mar-2023-Sample#1898'
csv_and_code_abs_directory = '/Users/bashit.a/Documents/Alzheimer/Codes/'

dropdown_name, dropdown_name_list, cwd, exp_folder = change_python_path(dropdown_name = default_sample_dir, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)

Python directory is set to load samples from : Mar-2023-Sample#1898


In [19]:
%matplotlib widget
heatmap_labels_file = "/Users/bashit.a/Documents/Alzheimer/heatmap_labels.csv"

annotation = [['1898_CING-roi0_0_0_masked_intp.h5', [np.arange(0,16), np.arange(225,246), np.arange(246,273), np.arange(381,409), np.arange(492, 502), np.arange(655,686), np.arange(656, 668), np.arange(681, 695), np.arange(738, 751), np.arange(820, 835), np.arange(1191, 1205), np.arange(1341, 1351), np.arange(1421, 1430), np.arange(1524, 1551),], 1],\
             ['1898_EC-roi0_0_0_masked_intp.h5', [np.arange(14, 43), np.arange(2388, 2419), np.arange(3295, 3323),], 1], \
             ['1898_EC-roi0_0_0_masked_intp.h5', [np.array([2868, 2870, 2744, 2745, 2129, 1960, 2185, 2158, 3181, 3180,])], 0,] \
             ]

df         = pd.DataFrame([], columns=['file', 'frame', 'merged', 'label'])
df.to_csv(heatmap_labels_file, index=False)


data    = pd.DataFrame([], columns = np.arange(0,len(qgrid2)))
labels  = np.array([])

In [20]:
%matplotlib widget

idx = 0
for item in annotation:
    file    = item[0]
    frames  = [j for i in item[1] for j in i]
    label   = item[2]
    directory = cwd
    Iq = read_Iq(file, scattering="merged", directory=directory)
    for fr  in frames:
        df.at[idx, 'file'] = file
        df.at[idx, 'frame'] = fr
        df.at[idx, 'merged'] = Iq[fr]
        df.at[idx, 'label']  = label
        
        data.at[idx,:]    = Iq[fr]
        labels = np.append(labels, label)
        idx = idx+1

In [21]:
idx_l , idx_u = qgrid_to_indices(qgrid2, 0.6) , qgrid_to_indices(qgrid2, 2)

n_points = len(np.arange(idx_l , idx_u))

data = data.values[:,idx_l:idx_u]
data = data.astype(np.float64)

In [22]:
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

In [23]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

In [24]:
train_labels = train_labels.astype(bool)  
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [25]:
plt.figure()
plt.grid()
plt.plot(qgrid2[idx_l:idx_u], normal_train_data[0])
plt.title("A Normal Tissue")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plt.figure()
plt.grid()
plt.plot(qgrid2[idx_l:idx_u], anomalous_train_data[0])
plt.title("An Anomalous Tissue")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
class AnomalyDetector(Model):
    def __init__(self):
        super(AnomalyDetector, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(32, activation="relu"),
            layers.Dense(16, activation="relu"),
            layers.Dense(8, activation="relu")])

        self.decoder = tf.keras.Sequential([
            layers.Dense(16, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(n_points, activation="sigmoid")])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = AnomalyDetector()

In [28]:
autoencoder.compile(optimizer='adam', loss='mae')

In [29]:
history = autoencoder.fit(normal_train_data, normal_train_data, 
          epochs=250, 
          batch_size=512,
          validation_data=(test_data, test_data),
          shuffle=True)

Epoch 1/250
1/1 [==============================] - 0s 466ms/step - loss: 0.2753 - val_loss: 0.2796
Epoch 2/250
1/1 [==============================] - 0s 20ms/step - loss: 0.2747 - val_loss: 0.2792
Epoch 3/250
1/1 [==============================] - 0s 19ms/step - loss: 0.2744 - val_loss: 0.2790
Epoch 4/250
1/1 [==============================] - 0s 19ms/step - loss: 0.2741 - val_loss: 0.2787
Epoch 5/250
1/1 [==============================] - 0s 22ms/step - loss: 0.2738 - val_loss: 0.2784
Epoch 6/250
1/1 [==============================] - 0s 19ms/step - loss: 0.2736 - val_loss: 0.2782
Epoch 7/250
1/1 [==============================] - 0s 17ms/step - loss: 0.2733 - val_loss: 0.2778
Epoch 8/250
1/1 [==============================] - 0s 19ms/step - loss: 0.2730 - val_loss: 0.2775
Epoch 9/250
1/1 [==============================] - 0s 18ms/step - loss: 0.2726 - val_loss: 0.2770
Epoch 10/250
1/1 [==============================] - 0s 20ms/step - loss: 0.2721 - val_loss: 0.2764
Epoch 11/250
1/1 [

In [30]:
plt.figure()
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
encoded_data = autoencoder.encoder(normal_test_data).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

plt.figure()
plt.plot(normal_test_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(qgrid2[idx_l:idx_u], decoded_data[0], normal_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
encoded_data = autoencoder.encoder(anomalous_test_data).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

plt.figure()
plt.plot(anomalous_test_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(qgrid2[idx_l:idx_u], decoded_data[0], anomalous_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.figure()
plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
threshold = np.mean(train_loss) + np.std(train_loss)/6
print("Threshold: ", threshold)

Threshold:  0.038093192813297115


In [59]:
reconstructions = autoencoder.predict(anomalous_test_data)
test_loss = tf.keras.losses.mae(reconstructions, anomalous_test_data)

plt.figure()
plt.hist(test_loss[None, :], bins=10)
plt.xlabel("Test loss")
plt.ylabel("No of examples")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
def predict(model, data, threshold):
    reconstructions = model(data)
    loss = tf.keras.losses.mae(reconstructions, data)
    return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
    print("Accuracy = {}".format(accuracy_score(labels, predictions)))
    print("Precision = {}".format(precision_score(labels, predictions)))
    print("Recall = {}".format(recall_score(labels, predictions)))

In [61]:
preds = predict(autoencoder, test_data, threshold)
print_stats(preds, test_labels)
tf.math.confusion_matrix(test_labels, preds)

Accuracy = 0.8260869565217391
Precision = 1.0
Recall = 0.8235294117647058


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 1,  0],
       [12, 56]], dtype=int32)>

In [62]:
preds = predict(autoencoder, anomalous_train_data, threshold)
print_stats(preds, train_labels[~train_labels])
tf.math.confusion_matrix(train_labels[~train_labels], preds)

Accuracy = 0.4444444444444444
Precision = 0.0
Recall = 0.0


/Users/bashit.a/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 5],
       [0, 0]], dtype=int32)>

In [64]:
predict(autoencoder, test_data, threshold)

<tf.Tensor: shape=(69,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True, False,  True, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True])>

In [65]:
test_labels

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])